In [ ]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')


cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = 274
        diff = 30
        l = [cc,dc]
        l.extend(price[274-diff:274])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_30.csv',index=False,header=None)

In [4]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20170901].reset_index(drop=True)
print(flow.shape)

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [cc,dc]
        l.extend(price[len(price)-diff:len(price)])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_30_3.csv',index=False,header=None)
1306

(26852, 6)
(17836, 6)
(13524, 32)
(98, 32)
[28.850751617386173, 29.606974506681816, 24.879545452559377, 28.32662537009235, 30.389340716045968]
28.41064753255314
(13524, 32)
(98, 32)
[44.05258824599091, 46.674349866095504, 41.395722106770236, 47.56227809803342, 43.38912490873209]
44.61481264512443
(13524, 32)
(98, 32)
[44.51432546180607, 48.44037792137856, 42.48764307387103, 47.259891579097406, 46.17399079589895]
45.77524576641041


[28.41064753255314, 44.61481264512443, 45.77524576641041]
39.60023531469599


In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20170801].reset_index(drop=True)
print(flow.shape)

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [cc,dc]
        l.extend(price[len(price)-diff:len(price)])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_30_2.csv',index=False,header=None)

(26852, 6)
(20874, 6)
(16562, 32)
(98, 32)
[25.53349701454074, 32.081066394307214, 33.05134084859181, 27.799265714166527, 27.48545272018824]
29.19012453835891
(16562, 32)
(98, 32)
[37.67321418905587, 41.7173275034044, 48.552122585253116, 44.89330268585487, 42.84634204475764]
43.13646180166518
(16562, 32)
(98, 32)
[40.673149019223565, 45.754343673122676, 47.52628017315651, 47.46147872669305, 43.662661359734834]
45.01558259038613


[29.19012453835891, 43.13646180166518, 45.01558259038613]
39.11405631013674
